# Advanced output handling


For grasping the basic idea of output handlers, check out this introduction: [validating agent output](validating_agent_output.html).

The purpose of this example is to demonstrate two concepts:

- Obtaining structured output from the agent
- Validating the output against the prompt that was given to the agent

This should give a good understanding of output handler semantics and prepare you to build your own ones.

In [1]:
from dotenv import load_dotenv
load_dotenv()

from motleycrew import MotleyCrew
from motleycrew.agents.langchain import ReActToolCallingMotleyAgent
from motleycrew.tasks import SimpleTask
from motleycrew.common.exceptions import InvalidOutput

from motleycrew.tools.code import PythonREPLTool
from motleycrew.tools.llm_tool import LLMTool
from motleycrew.tools import MotleyTool

In the [validating agent output](validating_agent_output.html) example, we used a simple tool to receive and check the output.

Here we want to have access to the agent input, so we subclass the `MotleyTool` class and implement its `run` method. Inside it, we'll call an LLM critic that will assess the agent's output.

`MotleyTool` has the `exceptions_to_reflect` argument, which allows us to reflect exceptions from the tool back to the agent. This is useful for providing feedback on why the tool failed and how to fix it. By default, it reflects `InvalidOutput` exceptions, but you can pass other exceptions to it as well, even the `Exception` class for reflecting any exception.

In [2]:
class CoderOutputHandler(MotleyTool):
    def __init__(self):
        super().__init__(
            name="coder_output_handler",
            description="Output handler. ONLY RETURN THE FINAL RESULT USING THIS TOOL!",
            return_direct=True,
            exceptions_to_reflect=[InvalidOutput, ValueError],
        )  # args_schema is inferred automatically from the run method signature, but you can specify it explicitly

    def run(self, code: str, comment: str):
        agent_prompt = self.agent_input["prompt"]
        critic = LLMTool(
            name="critic",
            description="improve the agent output",
            prompt=(
                "An AI agent was given this prompt: \n{agent_prompt}\n\n"
                "Here's the code it generated: \n```\n{code}\n```\n\n"
                "And the comment it gave: \n{comment}\n\n"
                "The code and the comment must be sensible and easily understandable. "
                "Give constructive advice on how to improve them unless they are already perfect. "
                "In this case, just return a single \"OK\" without quotes."
            )
        )

        critic_message = critic.invoke({"agent_prompt": agent_prompt, "code": code, "comment": comment}).content
        print("Critic response: ", critic_message)

        if critic_message.strip().lower().startswith("ok"):
            return code, comment
        raise InvalidOutput(critic_message)  # This will be returned to the agent to make it improve its output

In case the critic is not satisfied, the `InvalidOutput` exception will be returned to the agent, as if a regular tool was called. If everything is OK, we just return the agent's output.

The ReActToolCallingMotleyAgent constructor has a `max_iterations` argument, as well as most other agents. It's the maximum number of times the agent will be allowed to call tools. If the agent doesn't provide a valid output after `max_iterations`, an exception will be raised. To avoid this, you can add an iteration counter to the critic's state.

The output schema is inferred automatically from the `run` method signature. You can also specify it directly using the `args_schema` argument.

In [3]:
crew = MotleyCrew()

coder = ReActToolCallingMotleyAgent(
    name="coder",
    tools=[PythonREPLTool(), CoderOutputHandler()],
    force_output_handler=True,
    verbose=True,
    max_iterations=15,
)

task = SimpleTask(
    crew=crew,
    name="Explain and implement the bubble sorting algorithm",
    description="Write a Python program that implements the bubble sorting algorithm "
                "and test your implementation using the REPL tool. \n"
                "Return the code and a comment explaining how it works.",
    agent=coder,
)

In [4]:
crew.run()
code, comment = task.output



> Entering new AgentExecutor chain...



Invoking: `python_repl` with `{'command': 'def bubble_sort(arr):\n    n = len(arr)\n    for i in range(n):\n        for j in range(0, n-i-1):\n            if arr[j] > arr[j+1]:\n                arr[j], arr[j+1] = arr[j+1, arr[j]]\n    return arr\n\n# Test the bubble sort function\nsample_list = [64, 34, 25, 12, 22, 11, 90]\nsorted_list = bubble_sort(sample_list)\nprint(sorted_list)'}`
responded: Thought: The next step is to write the Python code for the bubble sort algorithm. After writing the code, I will test it using the REPL tool to ensure it works correctly. Finally, I will return the code along with a comment explaining how it works.

Let's start by writing the bubble sort algorithm and testing it with a sample list.



TypeError('list indices must be integers or slices, not tuple')
Invoking: `python_repl` with `{'command': 'def bubble_sort(arr):\n    n = len(arr)\n    for i in range(n):\n        for j in range(0, n-i-1):\n            if arr[j] > arr[j+1]:\n                arr[j

See, the initial code was optimized thanks to the critic!

In [5]:
from IPython.display import display, Markdown, Code

display(Code(code, language="python"))

def bubble_sort(array):
    """
    Sorts an array using the bubble sort algorithm.

    The bubble sort algorithm works by repeatedly stepping through the list to be sorted,
    comparing each pair of adjacent items and swapping them if they are in the wrong order.
    This process is repeated until the list is sorted. The algorithm gets its name because
    smaller elements 'bubble' to the top of the list.

    Parameters:
    array (list): The list of elements to be sorted.

    Returns:
    list: The sorted list.

    Time Complexity:
    - Worst and Average Case: O(n^2)
    - Best Case: O(n) when the list is already sorted

    Space Complexity:
    - O(1) as it is an in-place sorting algorithm
    """
    length = len(array)
    for i in range(length):
        swapped = False
        for j in range(0, length - i - 1):
            if array[j] > array[j + 1]:
                # Swap if the element found is greater than the next element
                array[j], array[j + 1] = array[j + 1], array[j]
                swapped = True
        # If no elements were swapped, the array is already sorted
        if not swapped:
            break
    return array

# Test the bubble sort function with various cases
test_cases = [
    [64, 34, 25, 12, 22, 11, 90],
    [],
    [1],
    [2, 1],
    [3, 3, 3],
    [5, 1, 4, 2, 8],
    [-1, -3, -2, -5, -4],
    [10, 9, 8, 7, 6, 5, 4, 3, 2, 1],
    [7, 7, 7, 7, 7]
]

for i, sample_list in enumerate(test_cases):
    sorted_list = bubble_sort(sample_list.copy())
    print(f"Test case {i + 1}: Original: {sample_list}, Sorted: {sorted_list}")

In [6]:
display(Markdown(comment))

The bubble sort algorithm works by repeatedly stepping through the list to be sorted, comparing each pair of adjacent items and swapping them if they are in the wrong order. This process is repeated until the list is sorted. The algorithm gets its name because smaller elements 'bubble' to the top of the list. The outer loop runs n times, and the inner loop runs n-i-1 times, where n is the length of the list and i is the current iteration of the outer loop. An optimization is added with a flag to detect if any swapping happened in the inner loop. If no swapping happened, the list is already sorted, and the loop can be terminated early. The time complexity of bubble sort is O(n^2) in the worst and average cases, and O(n) in the best case when the list is already sorted. The space complexity is O(1) as it is an in-place sorting algorithm.

In [7]:
final_result = f"{code}\n\n{comment}"